# Calculation of Costs Polygon Analysis


## EC2 - local

Instance type: r5.2xlarge

Price: 0,504 USD / H

### layers: 25

Max. Requests/s: <b>17.2</b>

### layers: 50

Max. Requests/s: <b>7.9</b>

In [26]:
requests_per_second_25_l = 17.2
requests_per_hour_25_l = requests_per_second_25_l * 60 * 60
costs_per_request_ec2_25_l = 0.504 / requests_per_hour_25_l
print(f'25 Layers, Costs per request: {costs_per_request_ec2_25_l:.20f}')

25 Layers, Costs per request: 0.00000813953488372093


In [27]:
requests_per_second_50_l = 7.9
requests_per_hour_50_l = requests_per_second_50_l * 60 * 60
costs_per_request_ec2_50_l = 0.504 / requests_per_hour_50_l
print(f'50 Layers, Costs per request: {costs_per_request_ec2_50_l:.20f}')

50 Layers, Costs per request: 0.00001772151898734177


### Lambda
Mean Execution Time Endpoint: ~ 460 ms

Mean Execution Time per layer: 30 ms

Number of layers: 25

Price: 0,0000166667 USD / Gigabyte-Second 

Price: 0,20 USD pro 1 Mio. Requests

Memory: 1024 MB == 1 GB



Api Gateway Price: 1,20 USD / Million

In [28]:
# Constant Variables 
LAMBDA_PRICE_PER_MS = 0.0000166667 / 1000 # GB can be omitted -> all functions run with 1 GB
LAMBDA_PRICE_PER_REQUEST = 0.2 / 1000000
API_GW_PRICE_PER_REQUEST = 1.2 / 1000000
STEP_FUNCTIONS_PRICE_PER_REQUEST = 0.000001
STEP_FUNCTIONS_PRICE_PER_MS = 0.0000001042 / 1000

In [29]:
def calc_total_step_functions_price(lambda_mean_execution_time_endpoint, lambda_mean_execution_time_per_layer, number_of_layers,
                                   step_functions_mean_execution_time, add_api_gw_price: bool = True):

    lambda_overall_execution_time = lambda_mean_execution_time_endpoint + (lambda_mean_execution_time_per_layer * number_of_layers)

    lambda_price = (lambda_overall_execution_time * LAMBDA_PRICE_PER_MS) + LAMBDA_PRICE_PER_REQUEST
    
    if add_api_gw_price:
        lambda_price += API_GW_PRICE_PER_REQUEST

    step_functions_price = (step_functions_mean_execution_time * STEP_FUNCTIONS_PRICE_PER_MS) + STEP_FUNCTIONS_PRICE_PER_REQUEST

    total_price = lambda_price + step_functions_price
    
    return total_price

In [30]:
# Execution Variables
lambda_mean_execution_time_endpoint = 460 
lambda_mean_execution_time_per_layer = 30 
number_of_layers = 25
step_functions_mean_execution_time = 179

In [31]:
lambda_total_price_25_layers = calc_total_step_functions_price(lambda_mean_execution_time_endpoint, lambda_mean_execution_time_per_layer, 25,
                                                        step_functions_mean_execution_time)

lambda_total_price_50_layers = calc_total_step_functions_price(lambda_mean_execution_time_endpoint, lambda_mean_execution_time_per_layer, 50,
                                                        step_functions_mean_execution_time)

In [32]:
print(f'25 Layers, Costs per request: {lambda_total_price_25_layers:.20f}')
print(f'50 Layers, Costs per request: {lambda_total_price_50_layers:.20f}')

25 Layers, Costs per request: 0.00002258535880000000
50 Layers, Costs per request: 0.00003508538380000000


## Ratio

In [33]:
lambda_total_price_25_layers / costs_per_request_ec2_25_l 

2.7747726525714285

Result: Lambda price 2.77 times higher than ec2 for 25 Layers

In [34]:
lambda_total_price_50_layers / costs_per_request_ec2_50_l

1.979818085857143

Result: Lambda price 1.97 times higher than ec2  for 25 Layers